In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#import plotly.graph_objects as go
from datetime import datetime
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
%matplotlib inline
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
"""
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
"""

C:\Users\taro\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\taro\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\Users\taro\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\taro\Anaconda3\lib\site-packages\sklearn\linear_model\lea

'\nimport torch\nfrom torch import nn\nfrom torch.utils.data import DataLoader\nfrom torchvision import datasets\nfrom torchvision.transforms import ToTensor, Lambda, Compose\n'

In [2]:
# path取得
cd_folder = os.getcwd()
work_folder = cd_folder + "\\dressipi_recsys2022"
print("cd: " + cd_folder)
print("work: " + work_folder)

cd: C:\Users\taro\Documents\Kaggle\Recsys
work: C:\Users\taro\Documents\Kaggle\Recsys\dressipi_recsys2022


# 0. 準備

## データ加工

Rawtable
- candidate_items: lb, finalで購入されるitem候補
- item_features：itemの特徴
- test_final_sessions: test用session(最終用）
- test_leaderboard_sessions: test用session(lb用）
- train_purchases: train用sessionと購入item_id
- train_sessions: train用session

疎行列（加工）
- train_sessions_cand_sparse: train用sessionのうち，購入item_idがcandidate_itemsに含まれる
- train_purchases_not_cand_sparse: train用sessionのうち，購入item_idがcandidate_itemsに含まれない
- test_leaderboard_sessions_sparse: test用session(lb用)
- test_final_sessions_sparse: test用session(最終用）

In [3]:
class DataLoader:
    
    def __init__(self, data_path = work_folder):
        self.data_path = data_path
    
    # RawTable取得
    def load_rawtable(self):
        
        ### rawdata
        candidate_items = pd.read_csv( work_folder + "\\" + "candidate_items.csv")
        item_features = pd.read_csv( work_folder + "\\" + "item_features.csv")
        test_final_sessions = pd.read_csv( work_folder + "\\" + "test_final_sessions.csv")
        test_leaderboard_sessions = pd.read_csv( work_folder + "\\" + "test_leaderboard_sessions.csv")
        train_purchases = pd.read_csv( work_folder + "\\" + "train_purchases.csv")
        train_sessions = pd.read_csv( work_folder + "\\" + "train_sessions.csv")
        
        return candidate_items, item_features, test_final_sessions, test_leaderboard_sessions, train_purchases, train_sessions
    
    # RawTable -> 疎行列
    def load_sparsetable(self):
        
        candidate_items, item_features, test_final_sessions, test_leaderboard_sessions, train_purchases, train_sessions = self.load_rawtable()
        
        ### 疎行列作成
        # train_dataのうち，購入itemがcandidate_itemに含まれるsessionを抽出
        candidate_items_flg = candidate_items['item_id'].unique()
        candidate_session_flg = train_purchases[train_purchases['item_id'].isin(candidate_items_flg)]['session_id'].values

        # train_dataのうち購入itemがcandidate_itemに含まれるsession
        train_purchases_cand = train_purchases[train_purchases['session_id'].isin(candidate_session_flg)]
        train_sessions_cand  = train_sessions[train_sessions['session_id'].isin(candidate_session_flg)]

        # train_dataのうち購入itemがcandidate_itemに含まれないsession
        train_purchases_not_cand = train_purchases[~train_purchases['session_id'].isin(candidate_session_flg)]
        train_sessions_not_cand  = train_sessions[~train_sessions['session_id'].isin(candidate_session_flg)]

        # 疎行列のvalue作成
        train_purchases_cand['count'] = 1
        train_sessions_cand['count'] = 1
        train_purchases_not_cand['count'] = 1
        train_sessions_not_cand['count'] = 1
        test_leaderboard_sessions['count'] = 1
        test_final_sessions['count'] = 1
        
        index = 'session_id'
        col = 'item_id'
        val = 'count'
        col_list = item_features['item_id'].unique()

        train_sessions_cand_sparse = self.make_sparse_pivot(train_sessions_cand, index, col, val, col_list = col_list)
        train_purchases_not_cand_sparse = self.make_sparse_pivot(train_purchases_not_cand, index, col, val, col_list = col_list)
        test_leaderboard_sessions_sparse = self.make_sparse_pivot(test_leaderboard_sessions, index, col, val, col_list = col_list)
        test_final_sessions_sparse = self.make_sparse_pivot(test_final_sessions, index, col, val, col_list = col_list)
        
        return train_sessions_cand_sparse, train_purchases_not_cand_sparse, test_leaderboard_sessions_sparse, test_final_sessions_sparse
    
    #参考URL: https://developers.microad.co.jp/entry/2019/05/10/180000
    #疎行列のpivotテーブルを作成
    def make_sparse_pivot(self, df, index= 'session_id', col = 'item_id', val = 'count', index_list = [], col_list = []): 
        if(len(index_list) == 0):
            index_list = df[index].unique()
        if(len(col_list) == 0):
            col_list = df[col].unique()
        index_categorical = pd.api.types.CategoricalDtype(categories=sorted(index_list), ordered=True)
        col_categorical = pd.api.types.CategoricalDtype(categories=sorted(col_list), ordered=True)
        row_num = df[index].astype(index_categorical).cat.codes
        col_num = df[col].astype(col_categorical).cat.codes

        sparse_matrix = csr_matrix((df[val], (row_num, col_num)), \
                               shape=(index_categorical.categories.size, \
                                      col_categorical.categories.size))
        return sparse_matrix
    
    # validationデータ作成(作成中,load_sparsetableをtrain用とtest用に分割してtrain用をsplit_dataに流用する, sparseのindexはどうとる?, )
    def split_data(self, df_session_sparse, test_size = 0.2, seed = 0):
        X_train, X_test = train_test_split(df_session_sparse, test_size = test_size, random_state = seed)
        return X_train, X_test

In [7]:
#データ読み込み
warnings.simplefilter('ignore')
data_loader = DataLoader(data_path = work_folder)
# rawtable（testの購入候補，itemの特徴，test_final, test_lb, train購入item, train）
candidate_items, item_features, test_final_sessions, test_leaderboard_sessions, train_purchases, train_sessions = data_loader.load_rawtable()
# 疎行列(trainのうち，testの購入候補が含まれるsession, trainのうち，testの購入候補が含まれないsession, test_lb, test_final)
train_sessions_cand_sparse, train_purchases_not_cand_sparse, test_leaderboard_sessions_sparse, test_final_sessions_sparse = data_loader.load_sparsetable()

## アルゴリズム実装（順次追加）

In [8]:
class Recommender:
    
    def KNN(self, df_train_sparse = train_sessions_cand_sparse, df_train_purchases = train_purchases, \
            item_features = item_features, \
            n_neighbors_fit = 10, df_test = test_leaderboard_sessions,  \
            df_test_sparse = test_leaderboard_sessions_sparse, n_neighbors_show = 10000):
        
        # KNNで訓練
        #評価:cos類似度, bruteは総当り方式。
        rec = NearestNeighbors(n_neighbors=n_neighbors_fit, algorithm= "brute", metric= "cosine")
        rec_model = rec.fit(df_train_sparse)
    
        #予測：60分くらい
        df_sub_lb = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
        item_id = []
        session_id = []
        rank = []
        count = 0
        
        unique_session_lb = df_test['session_id'].unique()                 #testのsession(unique)
        unique_item_id_list = item_features['item_id'].unique()            #全item_id(unique)
        df_train_purchases_ri = df_train_purchases.set_index('session_id') #trainの各sessionの購入item
        session_id_list = df_train_purchases_ri.index                      #trainのsession(unique)
        
        # testの各session毎に類似度上位n_neighbors_showの購入item，距離を取得．距離で重みづけをしてスコアが高い100アイテムを取得
        for i,s  in enumerate(unique_session_lb):
            #for文のカウント,50000で終わり
            if(count%1000 == 0):
                print(count)
            count += 1

            #対象のlb sessionとの距離が近い、上位num_n_neighborsのtrain session の、indexと距離を取得
            distance, indice = rec_model.kneighbors(df_test_sparse.getrow(i).toarray().flatten().reshape(1,-1),n_neighbors=n_neighbors_show)
            df_distance = pd.DataFrame(np.array([df_train_purchases_ri.loc[session_id_list[indice].flatten()].item_id.values, 1 - distance[0]]).T, columns=['candidate_item_id', 'distance'])

            #そのsessionに登場したアイテムを除く
            session_item_list = df_test[df_test['session_id']==s].item_id.values
            df_distance = df_distance[df_distance['candidate_item_id'].isin(session_item_list)==False]

            #距離をもとにした重みで、アイテムごとの加重平均をとる->スコアが高い上位100アイテムを取得
            prob_top100_item_id=df_distance.groupby('candidate_item_id').sum().sort_values('distance', ascending = 0).index.astype('int').values[0:100]

            #結果を格納していく
            item_id = np.concatenate([item_id,prob_top100_item_id])
            temp = np.array([s for i in range(len(prob_top100_item_id))])
            session_id = np.concatenate([session_id,temp])
            temp = np.array([i + 1 for i in range(len(prob_top100_item_id))])
            rank = np.concatenate([rank,temp])
            
            #提出用のデータを作成
            df_sub_lb = pd.DataFrame(np.vstack([session_id, item_id, rank]).T, columns=['session_id', 'item_id', 'rank'])
            df_sub_lb = df_sub_lb.astype('int')#intに明示的に変換しないと通らない
            
        return(df_sub_lb)

## 評価指標（未チェック）

In [9]:
class MetricCalculator:
    
    def calc(self, pred, y): #predはsession_id. item_id, rankの3列, yはsession_id, 

        mean_reciprocal_rank = self._calc_mean_reciprocal_rank()
        return mean_reciprocal_rank
    
    #!!正しいかチェックできていない!!
    def  _calc_mean_reciprocal_rank(self):
        
        Q = len(y)
        point = 0
        pred_session = pred["session_id"].unique().tolist()
        
        #predとyのlengthチェック
        if Q != len(pred_session):
            print("Error: len pred is not equal to len y")
            break
        #各sessionのpredにyのitem_idが含まれていた場合，predの1/rankを足し，最後にセッション数で除算
        for session in range(pred_session):
            pred_session = pred.loc[pred["session_id"]==session]
            y_session = y.loc[y["session_id"]==session]
            y_session_item = y_session["item_id"].value
            if y_session_item in pred_session["item_id"].tolist():
                point += 1.0 /(pred_session.loc[pred_session["item_id"]==y_session_item]["rank"].value)
            else:
                continue
        point = point / Q
        
        return point

# 1.計算実行

### 1.1. データの読み込み（上で読み込み済）

In [10]:
#data_loader = DataLoader(data_path = work_folder)

### rawtable（testの購入候補，itemの特徴，test_final, test_lb, train購入item, train）
#candidate_items, item_features, test_final_sessions, test_leaderboard_sessions, train_purchases, train_sessions = data_loader.load_rawtable()

### 疎行列(trainのうち，testの購入候補が含まれるsession, trainのうち，testの購入候補が含まれないsession, test_lb, test_final)
#train_sessions_cand_sparse, train_purchases_not_cand_sparse, test_leaderboard_sessions_sparse, test_final_sessions_sparse = data_loader.load_sparsetable()

### 1.2. アルゴリズム

In [11]:
warnings.simplefilter('ignore')
recommender = Recommender()
#df_train_sparse...trainの疎行列
#df_train_purchases...trainの購入データ
#item_features...(raw_table)
#n_neighbors_fit...学習用の近傍点
#df_test...testデータ
#df_test_sparse...testの疎行列
#n_neighbors_show...重みづけ計算
result_KNN = recommender.KNN(df_train_sparse = train_sessions_cand_sparse, df_train_purchases = train_purchases, \
            item_features = item_features, \
            n_neighbors_fit = 10, df_test = test_leaderboard_sessions,  \
            df_test_sparse = test_leaderboard_sessions_sparse, n_neighbors_show = 10000)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [12]:
result_KNN

,session_id,item_id,rank
0,26,22676,1
1,26,28067,2
2,26,13596,3
3,26,14550,4
4,26,4219,5
...,...,...,...
4999995,4439757,26433,96
4999996,4439757,648,97
4999997,4439757,17979,98
4999998,4439757,11891,99


### 1.3. 評価指標計算

In [13]:
pred = df_sub_lb
y = df_sub_lb #trainを分割している場合ここはvalidationの正解データを入れる
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(pred, y)
print(metrics)

実装中


### 1.4. アウトプット

In [16]:
#保存
savefile = result_KNN
file_name = "submit20220508_v1"
savefile.to_csv(cd_folder + "\\" + file_name + ".csv", index=False)

# 以下メモ

In [28]:
print(train_sessions_cand_sparse[0:5])

  (0, 13198)	1
  (1, 216)	1
  (1, 1725)	2
  (1, 3553)	1
  (1, 13739)	1
  (1, 21891)	1
  (1, 22292)	1
  (1, 23209)	1
  (2, 8756)	1
  (2, 14743)	1
  (2, 17332)	1
  (2, 18989)	1
  (3, 1806)	1
  (3, 5205)	1
  (3, 13271)	1
  (3, 14530)	1
  (3, 16667)	1
  (3, 17906)	1
  (4, 8798)	2
  (4, 13463)	1
  (4, 14028)	2
  (4, 16621)	3


In [29]:
train_sessions_cand_sparse
#train_purchases_not_cand_sparse

<450153x23691 sparse matrix of type '<class 'numpy.int64'>'
	with 1770447 stored elements in Compressed Sparse Row format>

In [86]:
test_leaderboard_sessions_sparse

<50000x23691 sparse matrix of type '<class 'numpy.int64'>'
	with 189283 stored elements in Compressed Sparse Row format>

In [30]:
# sparse行列のチェック
# https://note.nkmk.me/python-scipy-sparse-get-element-row-column/

csr_matrix(train_sessions_cand_sparse).getrow(2).toarray()

csr_matrix(train_sessions_cand_sparse).getrow(2).toarray().sum()

4

In [31]:
train_sessions_cand_sparse.getrow(0).toarray().flatten().reshape(1,-1)

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [169]:
test_id_list = test_leaderboard_sessions["session_id"].unique().tolist()

#trainと同列のデータ作成用
df_base_col = pd.DataFrame(np.array(col_list), columns = ["item_id"]).astype("category")
df_base_col["session_id"] = 99999999
df_base_col["count"] = 1
df_base_col = df_base_col[["session_id","item_id","count"]]

batch = 100
n_rep = len(test_id_list) // batch

#batchサイズ毎の処理
#for i_rep in range(n_rep):
for i_rep in range(2):
    
    # test_session_id取得（batchサイズ毎）
    if(i_rep == n_rep - 1):
        test_id_list_batch = test_id_list[ i_rep * batch : len(test_id_list) ]
    else:
        test_id_list_batch = test_id_list[ i_rep * batch : i_rep * batch + batch ]
        
    # 
    a = test_leaderboard_sessions.loc[test_leaderboard_sessions["session_id"].isin(test_id_list_batch)][["session_id","item_id","count"]]
    a["item_id"] = a["item_id"].astype("category")
    a = pd.concat([a,df_base_col])
    a = a.pivot_table(index = "session_id", columns = "item_id", values = "count", fill_value = 0)
    #a = a.loc[~a["session_id"]==99999999]

- sparseのindexはどのように取得する? -> splitで使用したい ->最終的には交差検証
- item情報どう入れる?
- candではないsessionは使用できる?
- KNN以外に良い方法がある?
- KNN高速化できる?
- Todo validationデータ作成して，KNNのパラメータを変えてみて精度の検証（上の疑問点は解決する必要あり）